# Dataset Cleaning Notebook - V5 (Excel Safe)

This notebook processes `Hackathon Challenge #1 Datasets Cleaned.csv` to produce `Hackathon_Datasets_Refined_v5.csv`.

### Improvements in V5:
-   **Excel Formula Safety:** Specifically removes leading `=`, `-`, `+` characters that cause `#NAME?` errors in Excel.
-   **Metadata Extraction:** Keeps the V4 logic for Job Level, Scope, and Internal status.

In [ ]:
import pandas as pd
import re
import os

# Configuration
INPUT_FILE = 'Hackathon Challenge #1 Datasets Cleaned.csv'
OUTPUT_FILE = 'Hackathon_Datasets_Refined_v5.csv'

In [ ]:
def clean_filename_v5(path):
    """
    Advanced cleaning with Excel safety.
    """
    if not isinstance(path, str):
        return ""

    # 1. Base filename
    filename = os.path.basename(path).replace('\\', '/')
    filename = filename.split('/')[-1]
    filename = os.path.splitext(filename)[0]

    # 2. Convert to Title Case
    filename = filename.title()

    # 3. Remove date prefixes
    filename = re.sub(r'^\d{4,8}[-_\s]*', '', filename)
    
    # 4. Remove common noise words
    noise_words = [
        r'\bPosting\b', r'\bJob Description\b', r'\bJd\b', 
        r'\bExternal\b', r'\bInternal\b', r'\bExpression Of Interest\b',
        r'\bSecondment\b', r'\bActing\b', r'\bTerm\b', r'\bContract\b',
        r'\bJob\b', r'\bAdvertisement\b'  # Added these based on screenshot artifacts
    ]
    
    for word in noise_words:
        filename = re.sub(word, '', filename, flags=re.IGNORECASE)

    # 5. EXCEL SAFETY FIX: Remove leading =, -, +, and whitespace
    # This fixes the "=-Manager" or "- Manager" issues causing #NAME?
    filename = re.sub(r'^[=\-+\s_]+', '', filename)

    # 6. Final whitespace clean
    filename = re.sub(r'\s+', ' ', filename).strip()
    
    return filename

def extract_metadata(row):
    """
    Extracts metadata and returns standardized columns.
    """
    full_title = clean_filename_v5(row.get('filename', ''))
    
    # Fallback to job_title if filename parse failed
    if len(full_title) < 3:
        # Also sanitize the fallback title just in case
        fallback = str(row.get('job_title', '')).title()
        full_title = re.sub(r'^[=\-+\s_]+', '', fallback).strip()

    # --- Level & Scope Extraction ---
    levels = ['Senior', 'Junior', 'Lead', 'Principal', 'Chief', 'Head Of', 'Director', 'Manager', 'Vice President', 'VP']
    found_level = ""
    for level in levels:
        if re.search(r'\b' + level + r'\b', full_title, re.IGNORECASE):
            found_level = level
            break
            
    scopes = ['Global', 'Regional', 'Local', 'Site']
    found_scope = ""
    for scope in scopes:
        if re.search(r'\b' + scope + r'\b', full_title, re.IGNORECASE):
            found_scope = scope
            break

    # Internal Status
    raw_text = str(row.get('filename', '')) + " " + str(row.get('job_title', ''))
    internal_keywords = ['internal', 'expression of interest', 'secondment', 'acting']
    is_internal = 'Yes' if any(k in raw_text.lower() for k in internal_keywords) else 'No'

    # Acronym Fixes
    acronyms = {"Hr": "HR", "It": "IT", "Hse": "HSE", "Vp": "VP", "Ceo": "CEO", "Cfo": "CFO"}
    words = full_title.split()
    fixed_words = [acronyms.get(w, w) for w in words]
    final_title = " ".join(fixed_words)

    return pd.Series([final_title, found_level, found_scope, is_internal])

In [ ]:
# Load Data
print(f"Loading data from {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE)

# Apply Logic
print("Processing titles...")
metadata_cols = ['Unified Job Title', 'Job Level', 'Scope', 'Internal Posting']
df[metadata_cols] = df.apply(extract_metadata, axis=1)

# Fill NaNs
for col in ['position_summary', 'responsibilities', 'qualifications']:
    if col in df.columns:
        df[col] = df[col].fillna("")

# Clean Up Columns
if 'filename' in df.columns:
    df = df.drop(columns=['filename'])

final_cols = metadata_cols + [c for c in df.columns if c not in metadata_cols]
df = df[final_cols]

# Export
print(f"Saving refined dataset to {OUTPUT_FILE}...")
df.to_csv(OUTPUT_FILE, index=False)

print("Done. First 5 rows:")
df.head()